欠損データ処理

In [1]:
import numpy as np
import pandas as pd

In [2]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

In [3]:
for dtype in ['object', 'int']:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype = object
10 loops, best of 5: 64.5 ms per loop

dtype = int
100 loops, best of 5: 2.25 ms per loop



objectは遅くなりがち

In [4]:
vals1.sum()

TypeError: ignored

In [5]:
vals2 = np.array([1, np.nan, 3, 4]) 
vals2.dtype

dtype('float64')

In [6]:
1 + np.nan

nan

In [7]:
0 *  np.nan

nan

In [8]:
vals2.sum(), vals2.min(), vals2.max()

(nan, nan, nan)

欠損値があっても計算可能な関数

In [9]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

NaNとNone

In [10]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

In [11]:
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int64

In [12]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

nanとNoneは基本的に同じものとして扱われる  
Noneを入れてもnanに変換される

null値

isnull(): null値か判定  
notnull(): isnullの逆  
dropna(): データからnull値を落とす  
fillna(): null値を埋める  
()の中に数値で埋める値、df.mean()とかdf.median()とかdf.mode()とか指定すれば平均値、中央値、最瀕値で埋められる  
method=で埋め方指定  
method='ffill':前の値  
method='bfill':後ろの値  

In [14]:
data = pd.Series([1, np.nan, 'hello', None])

In [15]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [16]:
data[data.notnull()]

0        1
2    hello
dtype: object

In [17]:
data.dropna()

0        1
2    hello
dtype: object

In [18]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [19]:
df.dropna()

,0,1,2
1,2.0,3.0,5


軸指定drop

In [20]:
df.dropna(axis='columns')

,2
0,2
1,5
2,6


In [21]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


how='all'ですべてnullの列・行だけを落とす

In [22]:
df.dropna(axis='columns', how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


thresh=<数値>で落とす行・列のnull以外の数の閾値を指定

In [23]:
df.dropna(axis='rows', thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


In [24]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [25]:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [26]:
# forward-fill
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [27]:
# back-fill
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

In [28]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


axisで軸の方向を指定

In [29]:
df.fillna(method='ffill', axis=1)

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0


In [30]:
# mean
df.fillna(df.mean())

,0,1,2,3
0,1.0,3.5,2,NaN
1,2.0,3.0,5,NaN
2,1.5,4.0,6,NaN
